In [ ]:
# !pip uninstall -y foresight
#!pip install git+https://github.com/SAGE-3/next.git@dev#subdirectory=foresight

In [ ]:
# !pip install langchain_openai
# !pip install langchain

In [1]:
import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")
# from src.config import config as conf, prod_type
from foresight.Sage3Sugar.pysage3 import PySage3
from langchain_openai import ChatOpenAI

In [2]:
prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}
# room_id = '09073d6d-6d79-4c98-bbea-4dd82106c098'
# board_id = '68993aeb-0ee6-4f95-b461-0f918fc7d969'
# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'

In [8]:
ps3 = PySage3(conf, prod_type)
cb = ps3.rooms['e3233abf-a47b-4c16-81ec-c6e985c67a5a'].boards['479d5114-a017-4b6e-973d-8933c46b8837']

In [ ]:
llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
llm_4o.invoke("hello")

Untitled.ipynb         _env.sh~               src
__init__.py            interactive_demo.ipynb
_env.sh                requirements.txt


In [37]:
parser_cls = "test_output_parser"
"".join(" ".join(parser_cls.split("_")).title().split())

'TestOutputParser'

In [66]:
import os
import json
import importlib
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

class PromptLoader:
    def __init__(self, base_dir="prompts"):
        self.base_dir = base_dir

    def _load_prompt_data(self, prompt_type, prompt_name):
        prompt_def_file_path = os.path.join(self.base_dir, prompt_type, f"{prompt_name}.json")
        with open(prompt_def_file_path, 'r') as file:
            prompt_data = json.load(file)
        return prompt_data

    def _generate_few_shot_prompt(self, prompt_data):
        kwargs = {
            
            "prefix": prompt_data['prefix'],
            
            "input_variables": prompt_data['input_variables'],
            "partial_variables": prompt_data.get('partial_variables', {}),
            "examples": prompt_data.get('examples_info', {}).get("examples_data", []),
            "example_prompt": prompt_data.get('examples_info', {}).get("examples_template", ""),
            "prefix": prompt_data.get('prefix', ""),
            "suffix": prompt_data.get('suffix', ""),
            "example_separator": prompt_data.get('examples_info', {}).get('example_separator', "\n")
        }

        if "format_instructions" in kwargs["partial_variables"]:
            parser_module_name = prompt_data['parser']
            module = importlib.import_module(f"src.prompts.parsers.{parser_module_name}")
            parser_cls_name = "".join(" ".join(parser_module_name.split("_")).title().split())
            ParserCls = getattr(module, parser_cls_name)
            output_parser = PydanticOutputParser(pydantic_object=ParserCls)
            kwargs["partial_variables"]["format_instructions"] = output_parser.get_format_instructions()
        examples = prompt_data.get('examples_info', {}).get("examples_data", [])
        example_prompt =  prompt_data.get('examples_info', {}).get("examples_template", "")
        examples_input_variables = prompt_data.get('examples_info', {}).get("examples_input_variables", [])
        if examples and example_prompt:
            examples_prompt = PromptTemplate(
                    input_variables=examples_input_variables,
                    template=example_prompt,
                )
            kwargs['examples'] = examples
            kwargs['example_prompt'] = examples_prompt
                    
        return FewShotPromptTemplate(**kwargs)

    def _generate_zero_shot_prompt(self, prompt_data):
        # Assuming zero-shot prompts require fewer parameters, adjust accordingly
        return FewShotPromptTemplate(
            input_variables=prompt_data['input_variables'],
            prefix=prompt_data.get('prefix', ""),
            suffix=prompt_data.get('suffix', "")
        )

    @staticmethod
    def create_prompt(base_dir, prompt_type, prompt_name):
        loader = PromptLoader(base_dir)
        data = loader._load_prompt_data(prompt_type, prompt_name)
        if prompt_type == "few_shot":
            return loader._generate_few_shot_prompt(data)
        elif prompt_type == "zero_shot":
            return loader._generate_zero_shot_prompt(data)
        else:
            raise ValueError(f"Invalid prompt type: {prompt_type}")

In [85]:
example_prompt = PromptLoader.create_prompt("src/prompts/", "few_shot", "select_apps")
example_prompt

originalItems: {originalItems}

query: {query}

```json


FewShotPromptTemplate(input_variables=['originalItems', 'query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"uuids": {"title": "Uuids", "description": "A valid Python list of UUIDs selected", "type": "array", "items": {"type": "string", "format": "uuid"}}}, "required": ["uuids"]}\n```'}, examples=[{'originalItems': '[{"app_id": "ba88e0df-d885-4ce3-ad74-006fbba69b43", "data": {"position": {"x": 1502426, "y": 1500697, "z": 0}, "size": {"width": 400, "height": 517, "depth": 0}, "rotation": {"x": 0, "y": 0, "z": 0}, "type":

In [82]:
output_end = ': Yo, my name is Jason\n```json'
example_prompt.format(greeting="Yo, my name is Jason")[-len(output_end):] == ": Yo, my name is Jason\n```json"

True

In [76]:
x = {'input_variables': ['originalItems', 'query'], 'partial_variables': {'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"uuids": {"title": "Uuids", "description": "A valid Python list of UUIDs selected", "type": "array", "items": {"type": "string", "format": "uuid"}}}, "required": ["uuids"]}\n```'}, 'examples': [{'originalItems': '[{"app_id": "ba88e0df-d885-4ce3-ad74-006fbba69b43", "data": {"position": {"x": 1502426, "y": 1500697, "z": 0}, "size": {"width": 400, "height": 517, "depth": 0}, "rotation": {"x": 0, "y": 0, "z": 0}, "type": "PDFViewer"}, "state": {"assetid": "4d2acd83-8fa2-4c7d-8e91-8e2e8cf45179", "currentPage": 0, "numPages": 8, "displayPages": 1, "analyzed": "", "client": ""}}, {"app_id": "0fbbc02b-6f62-4bbe-8bfb-b6698b7da401", "data": {"position": {"x": 1502778, "y": 1500423, "z": 0}, "size": {"width": 400, "height": 420, "depth": 0}, "rotation": {"x": 0, "y": 0, "z": 0}, "type": "Stickie"}, "state": {"text": "Philosophy", "fontSize": 36, "color": "yellow"}}, {"app_id": "9ae9a2eb-d901-44c8-83e3-5b447043fd67", "data": {"position": {"x": 1503749, "y": 1500342, "z": 0}, "size": {"width": 400, "height": 420, "depth": 0}, "rotation": {"x": 0, "y": 0, "z": 0}, "type": "Stickie"}, "state": {"text": "Tech and AI", "fontSize": 36, "color": "red"}}]', 'query': 'Which stickies are red', 'matchedItems': '["9ae9a2eb-d901-44c8-83e3-5b447043fd67"]'}], 'example_prompt': 'originalItems: {originalItems}\n\nquery: {query}\n\n```\njson\n{{{{\n  "uuids": {matchedItems}\n}}}}\n```', 'prefix': '', 'suffix': 'originalItems: {originalItems}\n\nquery: {query}\n\n```json', 'example_separator': '\n'}

In [73]:
x["example_prompt"]

'originalItems: {originalItems}\n\nquery: {query}\n\n```\njson\n{{{{\n  "uuids": {matchedItems}\n}}}}\n```'

In [ ]:
x[]